# Perform segmentation and feature extraction for each plate using CellProfiler Parallel

## Import libraries

In [2]:
import pathlib
import pprint

import sys

sys.path.append("/data/brussel/vo/000/bvo00026/vsc11013/Projects/IA-Yuwei-Bodipy-Pipeline/utils")
import cp_parallel
from cp_sequential import rename_sqlite_file

## Set paths and variables

In [4]:
# set the run type for the parallelization
run_name = "analysis"

# set main output dir for all plates
output_dir = pathlib.Path("./analysis_output")
output_dir.mkdir(exist_ok=True)

# directory where images are located within folders
images_dir = pathlib.Path("/data/brussel/vo/000/bvo00026/vsc11013/Projects/IA-Yuwei-Bodipy-Pipeline/cp_data")

# list for plate names based on folders to use to create dictionary
plate_names = []
# iterate through 0.download_data and append plate names from folder names that contain image data from that plate
for file_path in pathlib.Path("/data/brussel/vo/000/bvo00026/vsc11013/Projects/IA-Yuwei-Bodipy-Pipeline/cp_data").iterdir():
    if str(file_path.stem).startswith("Plate"):
        plate_names.append(str(file_path.stem))

print(plate_names)

['Plate5']


## Create dictionary with all info for each plate

In [5]:
plate_info_dictionary = {}

for name in plate_names:
    # Skip plates that already have sqlite outputs
    if any((output_dir / name).glob("*.sqlite")):
        continue

    # Find folder/file containing the plate name
    matches = list(images_dir.rglob(f"{name}"))
    if not matches:
        print(f"[WARN] No folder/file matching '{name}' under {images_dir}")
        continue

    path_to_images = matches[0].resolve(strict=True)

    plate_info_dictionary[name] = {
        "path_to_images": path_to_images,
        "path_to_output": (output_dir / name),
        "path_to_pipeline": pathlib.Path("./eHeps.cppipe").resolve(strict=True),
    }

pprint.pprint(plate_info_dictionary, indent=4)

{   'Plate5': {   'path_to_images': PosixPath('/vscmnt/brussel_pixiu_data/_data_brussel/vo/000/bvo00026/vsc11013/Projects/IA-Yuwei-Bodipy-Pipeline/cp_data/Plate5'),
                  'path_to_output': PosixPath('analysis_output/Plate5'),
                  'path_to_pipeline': PosixPath('/vscmnt/brussel_pixiu_data/_data_brussel/vo/000/bvo00026/vsc11013/Projects/IA-Yuwei-Bodipy-Pipeline/cp_analysis/eHeps.cppipe')}}


## Run analysis pipeline on each plate in parallel

This cell is not finished to completion due to how long it would take. It is ran in the python file instead.

In [ ]:
# Process data with cp_parallel
cp_parallel.run_cellprofiler_parallel(
    plate_info_dictionary=plate_info_dictionary, run_name=run_name
)

# rename the sqlite files to match the plate names
for name in plate_names:
    rename_sqlite_file(pathlib.Path(f"{output_dir}/{name}"), name)